In [1]:
from pymongo import MongoClient

In [2]:
# client = MongoClient()

In [4]:
client = MongoClient("mongodb://localhost:27017")

In [5]:
db = client.lietuvos_imones

In [7]:
db = client["lietuvos_imones"]

In [10]:
imones = db.imones

In [11]:
imones

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'lietuvos_imones'), 'imones')

In [13]:
nauja_imone = {"pavadinimas":"UAB nauja imones"}

In [14]:
result = imones.insert_one(nauja_imone)

In [16]:
result.inserted_id

ObjectId('6268e4d7bdb53e7844c9a605')

In [17]:
nauja_imone_1 = {"pavadinimas":"UAB nauja imone 1"}
nauja_imone_2 = {"pavadinimas":"UAB nauja imone 2"}

In [18]:
new_result = imones.insert_many([nauja_imone_1, nauja_imone_2])

In [19]:
new_result.inserted_ids

[ObjectId('6268e510bdb53e7844c9a606'), ObjectId('6268e510bdb53e7844c9a607')]

In [20]:
import pprint

In [28]:
imones.find_one({"pavadinimas": "UAB nauja imones"})

{'_id': ObjectId('6268e4d7bdb53e7844c9a605'),
 'pavadinimas': 'UAB nauja imones'}

In [29]:
imones.find_one({"pavadinimas": 'UAB "OLAN LT"'})


{'_id': ObjectId('5fb07731261973556a11202b'),
 'pavadinimas': 'UAB "OLAN LT"',
 'kategorijos': ['Naudingosios iškasenos',
  'Akmens gaminiai',
  'Betonas ir betono gaminiai',
  'Kelių tiesimas, remontas, tiltai',
  'Statybinės medžiagos',
  'Statybos technika, įranga, nuoma'],
 'įmonės_kodas': '302583298',
 'pvm_mokėtojo_kodas': 'LT100005854919',
 'vadovas': 'Rytis Ratomskis, direktorius',
 'adresas': 'Ateities pl. 31, LT-52167 Kaunas',
 'telefonas': '+370 643 14922',
 'mobilus_telefonas': '+370 643 45605',
 'faksas': '+370 37 210757',
 'tinklalapis': 'http://www.olan.lt',
 'darbuotojai': '6 darbuotojai (apdraustieji)',
 'sd_draudėjo_kodas': '2088490',
 'vidutinis_atlyginimas': '625,89 € (2020 m. rugsėjis)',
 'vsd_įmokų_suma': '881,69 € (2020 m. rugsėjis)',
 'creditinfo_co': 'Creditinfo CO sistemoje skolų neregistruota',
 'pardavimo_pajamos': '2019: 2 000 001 - 3 000 000 € be PVM',
 'transportas': '2020-10: 4 automobiliai (priklauso)',
 'pranešimai': ['VMI: Įmonė, kuriai taikomos VMI i

In [33]:
imones_list = []
for doc in imones.find({'įmonės_amžius': '9 metai 10 mėnesių 1 diena'}):
    imones_list.append(doc)

In [35]:
imones_list[0]

{'_id': ObjectId('5fb07731261973556a11202b'),
 'pavadinimas': 'UAB "OLAN LT"',
 'kategorijos': ['Naudingosios iškasenos',
  'Akmens gaminiai',
  'Betonas ir betono gaminiai',
  'Kelių tiesimas, remontas, tiltai',
  'Statybinės medžiagos',
  'Statybos technika, įranga, nuoma'],
 'įmonės_kodas': '302583298',
 'pvm_mokėtojo_kodas': 'LT100005854919',
 'vadovas': 'Rytis Ratomskis, direktorius',
 'adresas': 'Ateities pl. 31, LT-52167 Kaunas',
 'telefonas': '+370 643 14922',
 'mobilus_telefonas': '+370 643 45605',
 'faksas': '+370 37 210757',
 'tinklalapis': 'http://www.olan.lt',
 'darbuotojai': '6 darbuotojai (apdraustieji)',
 'sd_draudėjo_kodas': '2088490',
 'vidutinis_atlyginimas': '625,89 € (2020 m. rugsėjis)',
 'vsd_įmokų_suma': '881,69 € (2020 m. rugsėjis)',
 'creditinfo_co': 'Creditinfo CO sistemoje skolų neregistruota',
 'pardavimo_pajamos': '2019: 2 000 001 - 3 000 000 € be PVM',
 'transportas': '2020-10: 4 automobiliai (priklauso)',
 'pranešimai': ['VMI: Įmonė, kuriai taikomos VMI i